In [1]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 65kB/s 
     |████████████████████████████████| 204kB 57.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=faf54af7311ad3104c898362fa139138e7fe81e179c46780f4c50490c54707ad
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark import *
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import udf

import time
import urllib.request as ur
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
conf = SparkConf()
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '12G')
        .set('spark.driver.memory', '12G')
        .set('spark.driver.maxResultSize', '12G')
        .set('spark.sql.shuffle.partitions', '1600'))
sc = SparkContext(conf = conf, master="local", appName="MtgAnalyse")
sc

<SparkContext master=local appName=MtgAnalyse>

In [0]:
#sc.stop()

In [4]:
from google.colab import files
### ne pas utiliser sous firefox
### On importe AllCards.json
AllCardFile = files.upload()

Saving AllCards.json to AllCards.json


In [5]:
!ls /content 
!pwd

AllCards.json  sample_data
/content


In [0]:
AllCardsSqlContext = SQLContext (sc)
#AllCardsPdDf = pd.read_json (io.BytesIO(AllCardFile["AllCards.json"]), orient='index')
AllCardsDf  = AllCardsSqlContext.read.json ("/content/AllCards.json")

In [0]:
url = "https://www.mtgjson.com/files/AllCards.json"
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}

request = urllib.request.Request(url, None, headers)
response = urllib.request.urlopen(request)
data = json.loads(response.read())

NameError: ignored

In [0]:
data

In [0]:
#
#
### A bit of pandas
#
#

In [0]:
AllCardsPdDf = pd.read_json (io.BytesIO(AllCardFile["AllCards.json"]), orient='index')

In [0]:
AllCardsPdDf

,colorIdentity,colorIndicator,colors,convertedManaCost,edhrecRank,faceConvertedManaCost,foreignData,hand,hasNoDeckLimit,isBuyABox,isDateStamped,isReserved,layout,leadershipSkills,legalities,life,loyalty,manaCost,mtgArenaId,mtgoFoilId,mtgoId,name,names,otherFaceIds,power,printings,purchaseUrls,rulings,scryfallOracleId,side,subtypes,supertypes,text,toughness,type,types,uuid
"""Ach! Hans, Run!""","[G, R]",NaN,"[G, R]",6.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,NaN,{},NaN,NaN,{2}{R}{R}{G}{G},NaN,NaN,NaN,"""Ach! Hans, Run!""",NaN,NaN,NaN,[UNH],{'cardmarket': 'https://mtgjson.com/links/6227...,[],a2c5ee76-6084-413c-bb70-45490d818374,NaN,[],[],"At the beginning of your upkeep, you may say ""...",NaN,Enchantment,[Enchantment],8f9a35d5-a3a2-556b-88da-6686da3aaa34
"""Rumors of My Death . . .""",[B],NaN,[B],3.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,NaN,{},NaN,NaN,{2}{B},NaN,NaN,NaN,"""Rumors of My Death . . .""",NaN,NaN,NaN,[UST],{'cardmarket': 'https://mtgjson.com/links/371e...,[],38bcba8b-2838-4ac8-9976-f9ccaa94fdba,NaN,[],[],"{3}{B}, Exile a permanent you control with a L...",NaN,Enchantment,[Enchantment],201f7042-dda1-53d6-a0bd-8225ba73f34a
1996 World Champion,"[B, G, R, U, W]",NaN,"[B, G, R, U, W]",5.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,NaN,{},NaN,NaN,{W}{U}{B}{R}{G},NaN,NaN,NaN,1996 World Champion,NaN,NaN,NaN,[PCEL],NaN,[],39302443-4614-4e62-832a-b74eb2136bc7,NaN,[Legend],[],Cannot be the target of spells or effects. Wor...,NaN,Summon — Legend,[Summon],6b0e48c0-441d-5a62-a558-3c99473e4387
A Display of My Dark Power,[],NaN,[],0.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,scheme,NaN,{},NaN,NaN,NaN,NaN,NaN,NaN,A Display of My Dark Power,NaN,NaN,NaN,[OARC],NaN,"[{'date': '2010-06-15', 'text': 'The ability a...",156107a0-1657-4d04-904c-f8d19df32ff8,NaN,[],[],"When you set this scheme in motion, until your...",NaN,Scheme,[Scheme],5ccf5ac6-84e6-5ae8-a801-6e9cac2f1996
A Good Thing,"[B, W]",NaN,"[B, W]",6.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,NaN,{},NaN,NaN,{4}{W}{B},NaN,NaN,NaN,A Good Thing,NaN,NaN,NaN,[CMB1],{'tcgplayer': 'https://mtgjson.com/links/f4acf...,"[{'date': '2019-11-12', 'text': 'To double a p...",9cfdc604-71de-44e4-8ba7-2a3ee03aab68,NaN,[],[],Spells and abilities you control can't destroy...,NaN,Enchantment,[Enchantment],5e77de09-7521-54b6-b516-b7ed7740f7fe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zurgo Helmsmasher,"[B, R, W]",NaN,"[B, R, W]",5.0,3269.0,NaN,"[{'language': 'Japanese', 'name': '兜砕きのズルゴ', '...",NaN,NaN,NaN,NaN,NaN,normal,"{'brawl': False, 'commander': True, 'oathbreak...","{'commander': 'Legal', 'duel': 'Legal', 'legac...",NaN,NaN,{2}{R}{W}{B},NaN,NaN,NaN,Zurgo Helmsmasher,NaN,NaN,7,"[DDN, KTK, PKTK, PRM]",{'cardmarket': 'https://mtgjson.com/links/9f5f...,"[{'date': '2014-09-20', 'text': 'You still cho...",6c48d888-9f5d-43f4-adbd-61dbdba09260,NaN,"[Orc, Warrior]",[Legendary],Haste\nZurgo Helmsmasher attacks each combat i...,2,Legendary Creature — Orc Warrior,[Creature],95affd9d-36c6-521b-a44a-09975c2c1a6d
"Zyym, Mesmeric Lord","[B, U]",NaN,"[B, U]",4.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,"{'brawl': False, 'commander': True, 'oathbreak...",{},NaN,NaN,{1}{U}{B}{B},NaN,NaN,NaN,"Zyym, Mesmeric Lord",NaN,NaN,3,[CMB1],{'tcgplayer': 'https://mtgjson.com/links/fe8ce...,"[{'date': '2019-11-12', 'text': 'While resolvi...",fbd4ffd6-1ffb-4822-9ee8-ee2133e03b4b,NaN,"[Vampire, Wizard]",[Legendary],"Flying, deathtouch\nWhen Zyym, Mesmeric Lord e...",4,Legendary Creature — Vampire Wizard,[Creature],0b6a65a0-69fd-59f7-8186-9e4c507343a8
Zzzyxas's Abyss,[B],NaN,[B],3.0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,normal,NaN,{},NaN,NaN,{1}{B}{B},NaN,NaN,NaN,Zzzyxas's Abyss,NaN,NaN,NaN,[UNH],{'cardmarket': 'https://mtgjson.com/links/9ccf...,[],0f863bb9-d73a-485b-b620-fb23510ff6d1,NaN,[],[],"At the beginning of your upkeep, destroy all n...",NaN,Enchantment,[Enchantment],d55dfa2f-5c3f-5070-8791-72aab11793b8
_____,[U],NaN,[U],2.0,NaN,NaN,[],NaN,

In [0]:
dj = pd.read_json('AllCards.json', orient='index')
for col in dj.columns.to_list():
  if dj[col].dtypes == 'object': 
    dj[col] = dj[col].astype(str)
AllCardsDj = AllCardsSqlContext.createDataFrame(dj)

In [9]:
AllCardsDj.show(10)

+--------------------+--------------+--------------------+-----------------+----------+---------------------+--------------------+----+--------------+---------+-------------+----------+------+----------------+--------------------+----+-------+---------------+----------+----------+-------+--------------------+-----+------------+-----+---------+--------------------+--------------------+--------------------+----+----------+----------+--------------------+---------+---------------+---------------+--------------------+
|       colorIdentity|colorIndicator|              colors|convertedManaCost|edhrecRank|faceConvertedManaCost|         foreignData|hand|hasNoDeckLimit|isBuyABox|isDateStamped|isReserved|layout|leadershipSkills|          legalities|life|loyalty|       manaCost|mtgArenaId|mtgoFoilId| mtgoId|                name|names|otherFaceIds|power|printings|        purchaseUrls|             rulings|    scryfallOracleId|side|  subtypes|supertypes|                text|toughness|           typ

In [14]:
#Voyons si une couleur est favorisé
AllCardsDj.createOrReplaceTempView('AllCards')
proto = AllCardsSqlContext.sql("SELECT count(colorIdentity) as NB, colorIdentity FROM AllCards GROUP BY colorIdentity")
proto.show(truncate = False)

+----+-------------------------+
|NB  |colorIdentity            |
+----+-------------------------+
|200 |['R', 'W']               |
|3126|['B']                    |
|287 |['U', 'W']               |
|2   |['B', 'R', 'U', 'W']     |
|289 |['G', 'W']               |
|68  |['B', 'G', 'R', 'U', 'W']|
|3098|['W']                    |
|283 |['G', 'R']               |
|202 |['R', 'U']               |
|60  |['G', 'R', 'W']          |
|210 |['B', 'G']               |
|292 |['B', 'R']               |
|3104|['R']                    |
|30  |['B', 'G', 'U']          |
|292 |['B', 'U']               |
|25  |['B', 'G', 'W']          |
|2289|[]                       |
|58  |['B', 'G', 'R']          |
|24  |['G', 'R', 'U']          |
|2   |['G', 'R', 'U', 'W']     |
+----+-------------------------+
only showing top 20 rows



In [0]:
proto = AllCardsSqlContext.sql("SELECT name FROM AllCards WHERE colorIdentity = \"['U']\"")
proto.show(truncate = False)

+-------------------------+
|name                     |
+-------------------------+
|Abduction                |
|Aberrant Researcher      |
|Abjure                   |
|Aboshan's Desire         |
|Aboshan, Cephalid Emperor|
|Abstruse Interference    |
|Academy Drake            |
|Academy Elite            |
|Academy Journeymage      |
|Academy Researchers      |
|Academy Ruins            |
|Accumulated Knowledge    |
|Acid Rain                |
|Acquire                  |
|Adarkar Windform         |
|Admiral's Order          |
|Advanced Hoverguard      |
|Advanced Stitchwing      |
|Adverse Conditions       |
|Advice from the Fae      |
+-------------------------+
only showing top 20 rows



In [0]:
txtSql = "SELECT * FROM (SELECT count(colorIdentity) as NB, 'Black' as Color FROM AllCards WHERE colorIdentity LIKE '%B%'"
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Green' as Color FROM AllCards WHERE colorIdentity LIKE '%G%'"
txtSql += "UNION SELECT count(colorIdentity) as NB, 'White' as Color FROM AllCards WHERE colorIdentity LIKE '%W%'"
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Red' as Color FROM AllCards WHERE colorIdentity LIKE '%R%'"
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Blue' as Color FROM AllCards WHERE colorIdentity LIKE '%U%'"
txtSql += "UNION SELECT count(colorIdentity) as NB, 'None' as Color FROM AllCards WHERE colorIdentity LIKE '[]') ORDER BY NB"
proto = AllCardsSqlContext.sql(txtSql)
proto.show(truncate = False)
# il y a un léger favoritisme pour le noir

+----+-----+
|NB  |Color|
+----+-----+
|2289|None |
|4370|Green|
|4390|Blue |
|4407|White|
|4423|Red  |
|4463|Black|
+----+-----+



In [0]:
proto = AllCardsDj[['legalities']].groupby(['legalities']).count()
proto.show(truncate = False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|legalities                                                                                                                                                  |count|
+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|{'commander': 'Banned', 'duel': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'vintage': 'Legal'}                                                          |8    |
|{'commander': 'Legal', 'duel': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'pioneer': 'Legal', 'vintage': 'Legal'}                                       |2150 |
|{'brawl': 'Legal', 'commander': 'Legal', 'duel': 'Legal', 'historic': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'vintage': 'Legal'}                    |1    |
|{'command

In [0]:
proto = AllCardsSqlContext.sql("SELECT count(convertedManaCost) as NB, convertedManaCost FROM AllCards WHERE legalities LIKE  \"%'modern': 'Legal'%\" GROUP BY convertedManaCost ORDER BY convertedManaCost")
proto.show(truncate = False)

+----+-----------------+
|NB  |convertedManaCost|
+----+-----------------+
|491 |0.0              |
|1410|1.0              |
|2815|2.0              |
|3119|3.0              |
|2502|4.0              |
|1676|5.0              |
|978 |6.0              |
|424 |7.0              |
|145 |8.0              |
|54  |9.0              |
|28  |10.0             |
|10  |11.0             |
|6   |12.0             |
|1   |13.0             |
|1   |14.0             |
|2   |15.0             |
+----+-----------------+



In [0]:
proto = AllCardsSqlContext.sql("SELECT name, convertedManaCost FROM AllCards WHERE convertedManaCost IN(1000000,16,0.5)")
proto.show(truncate = False)
#Little girl fait référence à la plus jeune joueuse de magic

+-----------+-----------------+
|name       |convertedManaCost|
+-----------+-----------------+
|Draco      |16.0             |
|Gleemax    |1000000.0        |
|Little Girl|0.5              |
+-----------+-----------------+



In [0]:
# quel carte a la plus haute toughness ?
proto = AllCardsSqlContext.sql("SELECT MIN(name), COUNT(toughness) as NBtough, toughness FROM AllCards GROUP BY toughness ORDER BY NBtough,toughness")
proto.show(n=10000, truncate = False)
#les nan étant en grande partie des cartes non viable nous allons chercher à revenir sur les cartes viable avant de retourner sur la recherche de la carte la plus efficiente d'après le critère:


+------------------------------+-------+---------+
|min(name)                     |NBtough|toughness|
+------------------------------+-------+---------+
|Chimney Goyf                  |1      |*+1      |
|S.N.O.T.                      |1      |*²       |
|Ninja                         |1      |+0       |
|Half-Squirrel, Half-          |1      |-0       |
|Spinal Parasite               |1      |-1       |
|Little Girl                   |1      |.5       |
|Fraction Jackson              |1      |1.5      |
|Autochthon Wurm               |1      |14       |
|Impervious Greatwurm          |1      |16       |
|Vile Bile                     |1      |2.5      |
|Kharis & The Beholder         |1      |20       |
|Shapeshifter                  |1      |7-*      |
|B.F.M. (Big Furry Monster) (b)|1      |99       |
|Half-Orc, Half-               |2      |+1       |
|Emrakul, the Aeons Torn       |2      |15       |
|Angry Mob                     |2      |2+*      |
|Loopy Lobster                 

In [15]:
# Force + toughness - (MAX(convertedManaCost) - convertedManaCost)
txtSql = "SELECT a.name, (a.power + a.toughness - (m.maximumCC - a.convertedManaCost)) as effecience "
txtSql += " , a.power, a.convertedManaCost, a.toughness "
txtSql += " FROM AllCards a"
txtSql += " LEFT JOIN  (SELECT name, MAX(convertedManaCost) as maximumCC FROM AllCards GROUP BY name) m on m.name = a.name "
txtSql += " WHERE legalities LIKE  \"%'modern': 'Legal'%\"  "
txtSql += " ORDER BY effecience "
proto = AllCardsSqlContext.sql(txtSql)
proto.show(n=10000, truncate = False)

IllegalArgumentException: ignored

In [0]:
proto = AllCardsSqlContext.sql("SELECT name, (power + toughness - convertedManaCost) as effecience, power,convertedManaCost, toughness FROM AllCards WHERE legalities LIKE  \"%'modern': 'Legal'%\" AND power != 'nan' ORDER BY effecience DESC")
proto.show(n=10000, truncate = False)

+-------------------------------+----------+-----+-----------------+---------+
|name                           |effecience|power|convertedManaCost|toughness|
+-------------------------------+----------+-----+-----------------+---------+
|Death's Shadow                 |25.0      |13   |1.0              |13       |
|Ludevic's Abomination          |24.0      |13   |2.0              |13       |
|Impervious Greatwurm           |22.0      |16   |10.0             |16       |
|Worldspine Wurm                |19.0      |15   |11.0             |15       |
|Withengar Unbound              |19.0      |13   |7.0              |13       |
|Consulate Dreadnought          |17.0      |7    |1.0              |11       |
|Jokulmorder                    |17.0      |12   |7.0              |12       |
|Krosan Cloudscraper            |16.0      |13   |10.0             |13       |
|Ormendahl, Profane Prince      |16.0      |9    |0.0              |7        |
|Gigantosaurus                  |15.0      |10   |5.

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NB  |legalities                                                                                                                                                                                           |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|8   |{'commander': 'Banned', 'duel': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'vintage': 'Legal'}                                                                                                   |
|2150|{'commander': 'Legal', 'duel': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'pioneer': 'Legal', 'vintage': 'Legal'}                                                         

In [0]:
#On commence pour voir si le noir est toujours favoriser
txtSql = "SELECT NB, Color FROM (SELECT count(colorIdentity) as NB, 'Black' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '%B%' AND legalities LIKE  \"%'modern': 'Legal'%\" "
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Green' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '%G%' AND legalities LIKE  \"%'modern': 'Legal'%\" "
txtSql += "UNION SELECT count(colorIdentity) as NB, 'White' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '%W%' AND legalities LIKE  \"%'modern': 'Legal'%\" "
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Red' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '%R%' AND legalities LIKE  \"%'modern': 'Legal'%\" "
txtSql += "UNION SELECT count(colorIdentity) as NB, 'Blue' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '%U%' AND legalities LIKE  \"%'modern': 'Legal'%\" "
txtSql += "UNION SELECT count(colorIdentity) as NB, 'None' as Color, COUNT(legalities) FROM AllCards WHERE colorIdentity LIKE '[]' AND legalities LIKE  \"%'modern': 'Legal'%\") ORDER BY NB"
proto = AllCardsSqlContext.sql(txtSql)
proto.show(truncate = False)
# il y a un léger favoritisme pour le noir
# 3031 contre 4463

+----+-----+
|NB  |Color|
+----+-----+
|1264|None |
|2987|Blue |
|2990|Green|
|3016|White|
|3019|Red  |
|3031|Black|
+----+-----+



In [0]:
# On redémarre notre recherche de la carte la plus efficiente
proto = AllCardsSqlContext.sql("SELECT MIN(name), COUNT(toughness) as NBtough, toughness FROM AllCards WHERE legalities LIKE  \"%'modern': 'Legal'%\" GROUP BY toughness ORDER BY NBtough,toughness")
proto.show(truncate=False)

+-----------------------+-------+---------+
|min(name)              |NBtough|toughness|
+-----------------------+-------+---------+
|Spinal Parasite        |1      |-1       |
|Autochthon Wurm        |1      |14       |
|Impervious Greatwurm   |1      |16       |
|Emrakul, the Aeons Torn|2      |15       |
|Allosaurus Rider       |3      |1+*      |
|Ghalta, Primal Hunger  |4      |12       |
|Blightsteel Colossus   |7      |11       |
|Death's Shadow         |7      |13       |
|Bearer of the Heavens  |18     |10       |
|Ancient Brontodon      |20     |9        |
|Aggressive Mammoth     |54     |8        |
|Abominable Treefolk    |82     |*        |
|Altered Ego            |108    |0        |
|Accomplished Automaton |121    |7        |
|Abhorrent Overlord     |301    |6        |
|Abolisher of Bloodlines|586    |5        |
|Abomination of Gudul   |1037   |4        |
|Absolver Thrull        |1563   |3        |
|Abbot of Keral Keep    |1674   |1        |
|Abattoir Ghoul         |1935   